# 나이브 베이즈 완벽 프레임 - 시험용 v2 📊
## 모든 기출 유형 대응 가능! (F1 Score, ROC AUC, Accuracy 모두 포함)

## 1. 필수 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

# dmba 라이브러리 (있으면)
try:
    from dmba import classificationSummary
    HAS_DMBA = True
except:
    HAS_DMBA = False
    print("dmba 라이브러리가 없습니다. 기본 지표만 사용합니다.")

## 2. 데이터 로드 및 탐색

In [ ]:
# 데이터 로드 (⭐ 파일명 수정 필수!)
df = pd.read_csv('데이터파일.csv')

print("[데이터 정보]")
df.info()

print("\n[결측치 확인]")
print(df.isnull().sum())

print("\n[데이터 샘플]")
print(df.head())

In [ ]:
# 변수 타입 자동 분류
print("[변수 타입 분류]")
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"범주형 변수 ({len(categorical_cols)}개): {categorical_cols}")
print(f"수치형 변수 ({len(numerical_cols)}개): {numerical_cols}")

## 3. 데이터 전처리

In [ ]:
# 목표변수를 범주형으로 변환 (필요시)
# df['목표변수'] = df['목표변수'].astype('category')

# 독립변수/종속변수 분리 (⭐ 여기 수정 필수!)
outcome = '목표변수'  # ← 종속변수 이름 입력
predictors = [col for col in df.columns 
              if col not in [outcome, '제외할변수1', '제외할변수2']]

X = df[predictors]
y = df[outcome]

print(f"독립변수 개수: {len(predictors)}")
print(f"데이터 크기: {X.shape}")
print(f"\n목표변수 분포:\n{y.value_counts()}")

In [ ]:
# 나이브 베이즈 종류별 전처리 방법
print("\n[나이브 베이즈 3가지 유형]")
print("1. MultinomialNB: 카운트/빈도 데이터 (음수 불가, 정수 추천)")
print("2. GaussianNB: 연속형 데이터 (정규분포 가정)")
print("3. BernoulliNB: 이진 데이터 (0/1)")
print("\n⭐ 데이터 특성에 맞게 모델 선택!")

## 4. 전처리 - MultinomialNB용 (카운트/빈도 데이터)

In [ ]:
# MultinomialNB는 음수 값을 허용하지 않음
# 방법 1: 원-핫 인코딩 (drop_first=False 권장)
X_multi = pd.get_dummies(X, drop_first=False)

# 방법 2: 수치형 변수가 음수면 MinMaxScaler 사용
# scaler = MinMaxScaler()
# X_multi[numerical_cols] = scaler.fit_transform(X_multi[numerical_cols])

print(f"MultinomialNB용 변수 개수: {X_multi.shape[1]}")
print(f"변수 목록 (처음 10개): {X_multi.columns.tolist()[:10]}")

## 5. 전처리 - GaussianNB용 (연속형 데이터)

In [ ]:
# GaussianNB는 표준화 권장 (선택사항)
X_gauss = pd.get_dummies(X, drop_first=False)

# 수치형 변수 표준화
numerical_features = X_gauss.select_dtypes(include=['int64', 'float64']).columns.tolist()
if len(numerical_features) > 0:
    scaler = StandardScaler()
    X_gauss[numerical_features] = scaler.fit_transform(X_gauss[numerical_features])
    print(f"표준화된 수치형 변수: {len(numerical_features)}개")

print(f"GaussianNB용 변수 개수: {X_gauss.shape[1]}")

## 6. 데이터 분리

In [ ]:
# 방법 1: train_test_split 사용 (일반적)
# MultinomialNB용
train_X_multi, valid_X_multi, train_y, valid_y = train_test_split(
    X_multi, y, test_size=0.3, random_state=1, stratify=y
)

# GaussianNB용 (같은 분할 사용)
train_X_gauss, valid_X_gauss = X_gauss.iloc[train_X_multi.index], X_gauss.iloc[valid_X_multi.index]

print(f"학습 데이터: {train_X_multi.shape}")
print(f"검증 데이터: {valid_X_multi.shape}")
print(f"\n학습 데이터 클래스 분포:\n{train_y.value_counts()}")

In [ ]:
# 방법 2: 인덱스 기반 분리 (기출문제 스타일)
# 예: "0~1999번까지 학습, 2000번~ 테스트"

# train_X_multi = X_multi.iloc[:2000]
# train_X_gauss = X_gauss.iloc[:2000]
# train_y = y.iloc[:2000]
# valid_X_multi = X_multi.iloc[2000:]
# valid_X_gauss = X_gauss.iloc[2000:]
# valid_y = y.iloc[2000:]

# print(f"학습 데이터: {train_X_multi.shape}")
# print(f"검증 데이터: {valid_X_multi.shape}")

## 7. MultinomialNB - 기본 모델

In [ ]:
print("="*60)
print("MultinomialNB - 기본 모델 (alpha=1.0)")
print("="*60)

# 모델 생성 (alpha는 라플라스 스무딩)
nb_multi = MultinomialNB(alpha=1.0)
nb_multi.fit(train_X_multi, train_y)

# 예측
pred_train_y = nb_multi.predict(train_X_multi)
pred_valid_y = nb_multi.predict(valid_X_multi)

# 성과 측정
print("\n[학습 데이터 성과]")
if HAS_DMBA:
    classificationSummary(train_y, pred_train_y, class_names=nb_multi.classes_)
else:
    print(f"Accuracy:  {accuracy_score(train_y, pred_train_y):.4f}")
    print(f"Precision: {precision_score(train_y, pred_train_y, average='weighted'):.4f}")
    print(f"Recall:    {recall_score(train_y, pred_train_y, average='weighted'):.4f}")
    print(f"F1-Score:  {f1_score(train_y, pred_train_y, average='weighted'):.4f}")

print("\n[검증 데이터 성과]")
if HAS_DMBA:
    classificationSummary(valid_y, pred_valid_y, class_names=nb_multi.classes_)
else:
    print(f"Accuracy:  {accuracy_score(valid_y, pred_valid_y):.4f}")
    print(f"Precision: {precision_score(valid_y, pred_valid_y, average='weighted'):.4f}")
    print(f"Recall:    {recall_score(valid_y, pred_valid_y, average='weighted'):.4f}")
    print(f"F1-Score:  {f1_score(valid_y, pred_valid_y, average='weighted'):.4f}")

In [ ]:
# 🎯 기출 대응: 다양한 평가 지표로 교차검증
print("\n" + "="*60)
print("📊 5-Fold 교차검증 (다양한 평가 지표)")
print("="*60)

# Accuracy
scores_acc = cross_val_score(nb_multi, X_multi, y, cv=5, scoring='accuracy')
print(f"\n✅ 5-Fold CV Accuracy: {scores_acc.mean():.4f} (±{scores_acc.std():.4f})")

# F1 Score (기출에 자주 나옴!)
if len(np.unique(y)) == 2:
    scores_f1 = cross_val_score(nb_multi, X_multi, y, cv=5, scoring='f1')
    print(f"✅ 5-Fold CV F1 Score: {scores_f1.mean():.4f} (±{scores_f1.std():.4f})")
else:
    scores_f1 = cross_val_score(nb_multi, X_multi, y, cv=5, scoring='f1_weighted')
    print(f"✅ 5-Fold CV F1 Score (weighted): {scores_f1.mean():.4f} (±{scores_f1.std():.4f})")

# ROC AUC (이진 분류인 경우)
if len(np.unique(y)) == 2:
    scores_auc = cross_val_score(nb_multi, X_multi, y, cv=5, scoring='roc_auc')
    print(f"✅ 5-Fold CV ROC AUC: {scores_auc.mean():.4f} (±{scores_auc.std():.4f})")

## 8. GaussianNB - 기본 모델

In [ ]:
print("="*60)
print("GaussianNB - 기본 모델")
print("="*60)

# 모델 생성
nb_gauss = GaussianNB()
nb_gauss.fit(train_X_gauss, train_y)

# 예측
pred_train_y = nb_gauss.predict(train_X_gauss)
pred_valid_y = nb_gauss.predict(valid_X_gauss)

# 성과 측정
print("\n[학습 데이터 성과]")
if HAS_DMBA:
    classificationSummary(train_y, pred_train_y, class_names=nb_gauss.classes_)
else:
    print(f"Accuracy:  {accuracy_score(train_y, pred_train_y):.4f}")
    print(f"Precision: {precision_score(train_y, pred_train_y, average='weighted'):.4f}")
    print(f"Recall:    {recall_score(train_y, pred_train_y, average='weighted'):.4f}")
    print(f"F1-Score:  {f1_score(train_y, pred_train_y, average='weighted'):.4f}")

print("\n[검증 데이터 성과]")
if HAS_DMBA:
    classificationSummary(valid_y, pred_valid_y, class_names=nb_gauss.classes_)
else:
    print(f"Accuracy:  {accuracy_score(valid_y, pred_valid_y):.4f}")
    print(f"Precision: {precision_score(valid_y, pred_valid_y, average='weighted'):.4f}")
    print(f"Recall:    {recall_score(valid_y, pred_valid_y, average='weighted'):.4f}")
    print(f"F1-Score:  {f1_score(valid_y, pred_valid_y, average='weighted'):.4f}")

In [ ]:
# 🎯 기출 대응: 다양한 평가 지표로 교차검증
print("\n" + "="*60)
print("📊 5-Fold 교차검증 (다양한 평가 지표)")
print("="*60)

# Accuracy
scores_acc = cross_val_score(nb_gauss, X_gauss, y, cv=5, scoring='accuracy')
print(f"\n✅ 5-Fold CV Accuracy: {scores_acc.mean():.4f} (±{scores_acc.std():.4f})")

# F1 Score
if len(np.unique(y)) == 2:
    scores_f1_gauss = cross_val_score(nb_gauss, X_gauss, y, cv=5, scoring='f1')
    print(f"✅ 5-Fold CV F1 Score: {scores_f1_gauss.mean():.4f} (±{scores_f1_gauss.std():.4f})")
else:
    scores_f1_gauss = cross_val_score(nb_gauss, X_gauss, y, cv=5, scoring='f1_weighted')
    print(f"✅ 5-Fold CV F1 Score (weighted): {scores_f1_gauss.mean():.4f} (±{scores_f1_gauss.std():.4f})")

# ROC AUC (이진 분류인 경우)
if len(np.unique(y)) == 2:
    scores_auc_gauss = cross_val_score(nb_gauss, X_gauss, y, cv=5, scoring='roc_auc')
    print(f"✅ 5-Fold CV ROC AUC: {scores_auc_gauss.mean():.4f} (±{scores_auc_gauss.std():.4f})")

## 9. MultinomialNB alpha 튜닝

In [ ]:
# alpha 값에 따른 성능 변화
alphas = [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
train_scores = []
valid_scores = []
cv_scores = []

# 기출에서 자주 쓰는 지표 선택 (F1 Score 또는 ROC AUC)
if len(np.unique(y)) == 2:
    scoring = 'f1'  # 이진 분류: F1 Score
    metric_name = 'F1 Score'
else:
    scoring = 'f1_weighted'  # 다중 분류: F1 Score (weighted)
    metric_name = 'F1 Score (weighted)'

for alpha in alphas:
    nb = MultinomialNB(alpha=alpha)
    nb.fit(train_X_multi, train_y)
    
    # 학습/검증 데이터 점수
    train_scores.append(f1_score(train_y, nb.predict(train_X_multi), average='weighted' if len(np.unique(y)) > 2 else 'binary'))
    valid_scores.append(f1_score(valid_y, nb.predict(valid_X_multi), average='weighted' if len(np.unique(y)) > 2 else 'binary'))
    
    # 교차검증
    scores = cross_val_score(nb, X_multi, y, cv=5, scoring=scoring)
    cv_scores.append(scores.mean())

# 최적 alpha 찾기
best_alpha_idx = np.argmax(cv_scores)
best_alpha = alphas[best_alpha_idx]
best_score = cv_scores[best_alpha_idx]

print(f"\n최적 alpha: {best_alpha}")
print(f"최고 CV {metric_name}: {best_score:.4f}")

In [ ]:
# 시각화
plt.figure(figsize=(12, 6))
plt.plot(alphas, train_scores, 'o-', label='Train', linewidth=2)
plt.plot(alphas, valid_scores, 's-', label='Valid', linewidth=2)
plt.plot(alphas, cv_scores, '^-', label='5-Fold CV', linewidth=2)
plt.axvline(best_alpha, color='r', linestyle='--', label=f'Best alpha={best_alpha}')
plt.xscale('log')
plt.xlabel('alpha (Smoothing Parameter)', fontsize=12)
plt.ylabel(metric_name, fontsize=12)
plt.title('MultinomialNB Performance vs alpha', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. 최적 MultinomialNB 모델

In [ ]:
print("="*60)
print(f"최적 MultinomialNB (alpha={best_alpha})")
print("="*60)

nb_multi_best = MultinomialNB(alpha=best_alpha)
nb_multi_best.fit(train_X_multi, train_y)

# 예측
pred_train_y = nb_multi_best.predict(train_X_multi)
pred_valid_y = nb_multi_best.predict(valid_X_multi)

print("\n[학습 데이터 성과]")
if HAS_DMBA:
    classificationSummary(train_y, pred_train_y, class_names=nb_multi_best.classes_)
else:
    print(f"Accuracy:  {accuracy_score(train_y, pred_train_y):.4f}")
    print(f"F1-Score:  {f1_score(train_y, pred_train_y, average='weighted' if len(np.unique(y)) > 2 else 'binary'):.4f}")

print("\n[검증 데이터 성과]")
if HAS_DMBA:
    classificationSummary(valid_y, pred_valid_y, class_names=nb_multi_best.classes_)
else:
    print(f"Accuracy:  {accuracy_score(valid_y, pred_valid_y):.4f}")
    print(f"F1-Score:  {f1_score(valid_y, pred_valid_y, average='weighted' if len(np.unique(y)) > 2 else 'binary'):.4f}")

In [ ]:
# 🎯 기출 대응: 모든 주요 지표 출력
print("\n" + "="*60)
print("📊 최적 모델 교차검증 성과 (모든 지표)")
print("="*60)

# Accuracy
scores_acc_best = cross_val_score(nb_multi_best, X_multi, y, cv=5, scoring='accuracy')
print(f"\n✅ 5-Fold CV Accuracy: {scores_acc_best.mean():.4f} (±{scores_acc_best.std():.4f})")

# F1 Score
if len(np.unique(y)) == 2:
    scores_f1_best = cross_val_score(nb_multi_best, X_multi, y, cv=5, scoring='f1')
    print(f"✅ 5-Fold CV F1 Score: {scores_f1_best.mean():.4f} (±{scores_f1_best.std():.4f})")
else:
    scores_f1_best = cross_val_score(nb_multi_best, X_multi, y, cv=5, scoring='f1_weighted')
    print(f"✅ 5-Fold CV F1 Score (weighted): {scores_f1_best.mean():.4f} (±{scores_f1_best.std():.4f})")

# ROC AUC (이진 분류)
if len(np.unique(y)) == 2:
    scores_auc_best = cross_val_score(nb_multi_best, X_multi, y, cv=5, scoring='roc_auc')
    print(f"✅ 5-Fold CV ROC AUC: {scores_auc_best.mean():.4f} (±{scores_auc_best.std():.4f})")

## 11. 모델 비교 - MultinomialNB vs GaussianNB

In [ ]:
print("="*60)
print("🏆 모델 비교: MultinomialNB vs GaussianNB")
print("="*60)

# MultinomialNB
multi_acc = accuracy_score(valid_y, nb_multi_best.predict(valid_X_multi))
multi_f1 = f1_score(valid_y, nb_multi_best.predict(valid_X_multi), average='weighted' if len(np.unique(y)) > 2 else 'binary')
multi_cv_acc = scores_acc_best.mean()
multi_cv_f1 = scores_f1_best.mean()

# GaussianNB  
gauss_acc = accuracy_score(valid_y, nb_gauss.predict(valid_X_gauss))
gauss_f1 = f1_score(valid_y, nb_gauss.predict(valid_X_gauss), average='weighted' if len(np.unique(y)) > 2 else 'binary')
gauss_cv_acc = scores_acc.mean()
gauss_cv_f1 = scores_f1_gauss.mean()

comparison_df = pd.DataFrame({
    'Model': ['MultinomialNB', 'GaussianNB'],
    'Valid Accuracy': [multi_acc, gauss_acc],
    'Valid F1 Score': [multi_f1, gauss_f1],
    '5-Fold CV Accuracy': [multi_cv_acc, gauss_cv_acc],
    '5-Fold CV F1 Score': [multi_cv_f1, gauss_cv_f1]
})

# ROC AUC 추가 (이진 분류인 경우)
if len(np.unique(y)) == 2:
    multi_auc = roc_auc_score(valid_y, nb_multi_best.predict_proba(valid_X_multi)[:, 1])
    gauss_auc = roc_auc_score(valid_y, nb_gauss.predict_proba(valid_X_gauss)[:, 1])
    multi_cv_auc = scores_auc_best.mean()
    gauss_cv_auc = scores_auc_gauss.mean()
    
    comparison_df['Valid ROC AUC'] = [multi_auc, gauss_auc]
    comparison_df['5-Fold CV ROC AUC'] = [multi_cv_auc, gauss_cv_auc]

print("\n", comparison_df.to_string(index=False))

# 최고 성능 모델 (F1 Score 기준)
if multi_cv_f1 > gauss_cv_f1:
    print(f"\n✅ MultinomialNB가 더 우수 (CV F1 Score: {multi_cv_f1:.4f})")
    best_model = nb_multi_best
    best_model_name = 'MultinomialNB'
    best_valid_X = valid_X_multi
else:
    print(f"\n✅ GaussianNB가 더 우수 (CV F1 Score: {gauss_cv_f1:.4f})")
    best_model = nb_gauss
    best_model_name = 'GaussianNB'
    best_valid_X = valid_X_gauss

## 12. 로지스틱 회귀와 비교 (문제에서 요구시)

In [ ]:
# 로지스틱 회귀 모델
logreg = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)

# MultinomialNB와 같은 데이터로 학습
scores_logreg_f1 = cross_val_score(logreg, X_multi, y, cv=5, scoring=scoring)

print(f"로지스틱 회귀 5-Fold CV {metric_name}: {scores_logreg_f1.mean():.4f}")

# 비교
print(f"\n[모델 비교 - {metric_name}]")
print(f"MultinomialNB: {multi_cv_f1:.4f}")
print(f"GaussianNB:    {gauss_cv_f1:.4f}")
print(f"로지스틱 회귀:  {scores_logreg_f1.mean():.4f}")

## 13. 최종 성과 및 시각화

In [ ]:
# 이진 분류인 경우 ROC AUC 계산 및 시각화
if len(np.unique(y)) == 2:
    # MultinomialNB
    multi_pred_prob = nb_multi_best.predict_proba(valid_X_multi)[:, 1]
    multi_auc = roc_auc_score(valid_y, multi_pred_prob)
    fpr_multi, tpr_multi, _ = roc_curve(valid_y, multi_pred_prob)
    
    # GaussianNB
    gauss_pred_prob = nb_gauss.predict_proba(valid_X_gauss)[:, 1]
    gauss_auc = roc_auc_score(valid_y, gauss_pred_prob)
    fpr_gauss, tpr_gauss, _ = roc_curve(valid_y, gauss_pred_prob)
    
    # ROC 곡선
    plt.figure(figsize=(10, 8))
    plt.plot(fpr_multi, tpr_multi, color='blue', lw=2, 
             label=f'MultinomialNB (AUC = {multi_auc:.4f})')
    plt.plot(fpr_gauss, tpr_gauss, color='green', lw=2,
             label=f'GaussianNB (AUC = {gauss_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curve Comparison', fontsize=14)
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"MultinomialNB ROC AUC (검증): {multi_auc:.4f}")
    print(f"GaussianNB ROC AUC (검증): {gauss_auc:.4f}")
else:
    print("다중 분류: ROC 곡선은 이진 분류에서만 사용됩니다.")
    print("대신 Confusion Matrix를 확인하세요.")

In [ ]:
# Confusion Matrix (최고 성능 모델)
print(f"\n[Confusion Matrix - {best_model_name}]")
cm = confusion_matrix(valid_y, best_model.predict(best_valid_X))
print(cm)

# 시각화
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix - {best_model_name}')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Classification Report (최고 성능 모델)
print(f"\n[Classification Report - {best_model_name}]")
pred_y = best_model.predict(best_valid_X)
print(classification_report(valid_y, pred_y))

## 14. 최종 결과 요약 ✅

In [ ]:
print("="*60)
print("🎯 최종 결과 요약 (기출 대응 완료!)")
print("="*60)

final_acc = accuracy_score(valid_y, best_model.predict(best_valid_X))
final_f1 = f1_score(valid_y, best_model.predict(best_valid_X), average='weighted' if len(np.unique(y)) > 2 else 'binary')

if best_model_name == 'MultinomialNB':
    final_cv_acc = multi_cv_acc
    final_cv_f1 = multi_cv_f1
    print(f"""
✅ 최종 선택 모델: MultinomialNB
- 최적 alpha: {best_alpha}

📊 검증 데이터 성과:
- Accuracy:  {final_acc:.4f}
- F1 Score:  {final_f1:.4f}

📊 5-Fold CV 성과:
- Accuracy:  {final_cv_acc:.4f}
- F1 Score:  {final_cv_f1:.4f}
""")
else:
    final_cv_acc = gauss_cv_acc
    final_cv_f1 = gauss_cv_f1
    print(f"""
✅ 최종 선택 모델: GaussianNB

📊 검증 데이터 성과:
- Accuracy:  {final_acc:.4f}
- F1 Score:  {final_f1:.4f}

📊 5-Fold CV 성과:
- Accuracy:  {final_cv_acc:.4f}
- F1 Score:  {final_cv_f1:.4f}
""")

if len(np.unique(y)) == 2:
    final_auc = roc_auc_score(valid_y, best_model.predict_proba(best_valid_X)[:, 1])
    if best_model_name == 'MultinomialNB':
        final_cv_auc = multi_cv_auc
    else:
        final_cv_auc = gauss_cv_auc
    print(f"- Valid ROC AUC: {final_auc:.4f}")
    print(f"- 5-Fold CV ROC AUC: {final_cv_auc:.4f}")

print("\n🎉 분석 완료! 모든 기출 유형 대응 완료!")